In [ ]:
import s3fs
s3 = s3fs.S3FileSystem(anon=False)
import fwiVis.fwiVis as fv


from math import cos, asin, sqrt
import re
import dask ## Needed to install, but didn't update env

import numpy as np
import geopandas as gpd

import pandas as pd
from pandas.errors import SettingWithCopyWarning
from matplotlib import pyplot as plt
import os
import rioxarray as rio
import xarray as xr
import glob


from shapely.errors import ShapelyDeprecationWarning
from shapely.geometry import Point
import warnings

import datetime
from datetime import date
import traceback

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
warnings.filterwarnings("ignore", category=SettingWithCopyWarning) 
# SettingWithCopyWarning
#UserWarning




In [ ]:
### Get all IDs

flist = os.listdir("/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/WesternUS_REDO/2021/Largefire/")


import re

file_ids = [re.sub("([^a-zA-Z0-9].*)$", "", x) for x in flist]
file_ids = [re.sub("F", "", x) for x in file_ids]

file_ids = set(file_ids)
#file_ids
#print(len(file_ids))
#len(snap_2.index.unique()) ## Ok so the snapshot files are missing a fair amount.

In [ ]:
#file_ids

len(list(file_ids))

In [ ]:
## Read in the cheatsheet 

cheatsheet = pd.read_csv("/projects/my-public-bucket/fire_spread/CHEATSHEET_WesternUS_REDO_2020_2023-03-06.csv")
cheatsheet = cheatsheet.rename(columns={"0":"fireID",
                          "1": "Lat", 
                          "2": "Lon"})
cheatsheet = gpd.GeoDataFrame(cheatsheet, geometry=gpd.points_from_xy(cheatsheet["Lon"], cheatsheet["Lat"]))

,Unnamed: 0,0,1,2
0,0,7851,32.943797,-108.251687
1,1,11467,44.462707,-121.577746
2,2,8390,38.067751,-120.702455
3,3,12513,45.811285,-106.067291
4,4,6911,35.865107,-114.957638
...,...,...,...,...
417,417,12021,34.792345,-113.246040
418,418,12454,47.367430,-106.845373
419,419,10585,35.387976,-120.429968
420,420,247,31.806548,-102.066733


In [ ]:
## find all fireids in snapshot
pnw = gpd.read_file("/projects/my-public-bucket/fire_weather_vis/ref_data/NEON_DOMAIN/NEON_Domains.shp")# quickly subset by state to avoid computational stuff
pnw = pnw[pnw.DomainID == 16]

join = gpd.tools.sjoin(cheatsheet,pnw, predicate="within")
ids = join.fireID.unique()
## Get the IDs that are in the shapefile

In [16]:
ids

array([13203, 12959, 13051, 10491, 12984, 12961, 12953, 10532, 12981,
       10523, 13132, 13066, 13201, 13069, 12941, 12380, 12963, 13042,
       12985, 11440, 10261, 13525, 12960, 12779])

In [5]:
imerge_hr = xr.open_dataset("s3://veda-data-store-staging/EIS/zarr/FWI-GEOS-5-Hourly", engine="zarr")

In [6]:
#imerge_hr["GEOS-5_FWI"].to_dataframe
imerge_hr

# times = full_fr["t"].loc[full_fr["spread_line"] ]
# times_not = full_fr["t"].loc[full_fr["spread_line"] == False]
# times_not = times_not[(times_not.dt.hour == 13) & (times_not.dt.minute == 30)]


<xarray.Dataset>
Dimensions:      (time: 26880, lat: 533, lon: 1152)
Coordinates:
  * lat          (lat) float64 -58.0 -57.75 -57.5 -57.25 ... 74.5 74.75 75.0
  * lon          (lon) float64 -180.0 -179.7 -179.4 -179.1 ... 179.1 179.4 179.7
  * time         (time) datetime64[ns] 2020-01-01 ... 2023-01-24T23:00:00
Data variables:
    GEOS-5_FFMC  (time, lat, lon) float32 ...
    GEOS-5_FWI   (time, lat, lon) float32 ...
    GEOS-5_ISI   (time, lat, lon) float32 ...
Attributes: (12/46)
    Center:              NASA GISS / Columbia University
    Center::             center
    DCDryStartFactor:    5.0
    DCDryStartFactor::   DC number of days since precip mult factor for dry s...
    DCStart:             15.0
    DCStart::            DC starting value after wet winter
    ...                  ...
    snowCoverDaysCalc:   60.0
    snowCoverDaysCalc::  Number of days prior to spring over which to determi...
    startShutDays:       2.0
    startShutDays::      Number of previous days over which to consider start...
    tempThresh:          6.0
    tempThresh::         Temp thresh (C) to define start and end of winter

In [ ]:
imerge = xr.open_dataset("s3://veda-data-store-staging/EIS/zarr/GEOS5_FWI_GPM_LATE_v5_Daily.zarr", engine="zarr", chunks = "auto")
#imerge_year = imerge['GPM.LATE.v5_FWI'].loc["2020-01-01":"2020-10-31"]
imerge

In [ ]:
imerge.rio.write_crs("epsg:4326", inplace=True)
imerge.rio.set_spatial_dims(x_dim = "lon", y_dim = "lat", inplace = True)
#vars = ["GPM.LATE.v5_ISI","GPM.LATE.v5_FWI", "GPM.LATE.v5_FFMC"]
#imerge[vars].rio.set_spatial_dims(x_dim = "lon", y_dim = "lat", inplace = True)

In [ ]:
def load_large_fire(fireID, year = "2019", path_region = "WesternUS"):
    '''
    loads in largefire file based on fireID and layer, then preps it for "explore" by adding centriod data. Currently limited to one year. 
    
    INPUTS:
        
        fireID (str): fireID offire of interest. Can be found in gdf files read in by prep_gdf and load_file. Can be selected interactivly form a gdf if use gdf.explore()
        year (str): Year that fires took place. Default to 2019. Availible options differ by path_region. 
        path_region (str): This constructs the path that the fires are stored in. WesternUS and CONUS availible. 
    '''
    lf_files = glob.glob('/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/' + path_region +'/'+ year +'/Largefire/F' + fireID + '_*') 
    lf_ids = list(set([file.split('Largefire/')[1].split('_')[0] for file in lf_files])) 
    largefire_dict = dict.fromkeys(lf_ids)
    
    for lf_id in lf_ids:
         most_recent_file = [file for file in lf_files if lf_id in file][-1]
         largefire_dict[lf_id] = most_recent_file
    
    gdf = pd.concat([gpd.read_file(file,layer='perimeter') for key, file in largefire_dict.items()], 
                   ignore_index=True)
    gdf = gdf.to_crs('EPSG:4326')
    gdf['lon'] = gdf.centroid.x
    gdf['lat'] = gdf.centroid.y
    gdf['fireID'] = fireID
    gdf['func_failed'] = "None"
    gdf['log'] = "None"
    
    return gdf

In [ ]:
imerge = xr.open_dataset("s3://veda-data-store-staging/EIS/zarr/FWI-GEOS-5-Hourly", engine="zarr")
imerge = imerge["GEOS-5_FWI"]
imerge.rio.write_crs("epsg:4326", inplace=True)
imerge.rio.set_spatial_dims(x_dim = "lon", y_dim = "lat", inplace = True)

def imerge_merge(gdf, imerge, add_anomolies = False, **kwargs):
    
#     if (time_freq == "daily"):
#         imerge = xr.open_dataset("s3://veda-data-store-staging/EIS/zarr/GEOS5_FWI_GPM_LATE_v5_Daily.zarr", engine="zarr")
#         imerge.rio.write_crs("epsg:4326", inplace=True)
#         imerge.rio.set_spatial_dims(x_dim = "lon", y_dim = "lat", inplace = True)

#     if (time_freq == "hourly"):
#         imerge = xr.open_dataset("s3://veda-data-store-staging/EIS/zarr/FWI-GEOS-5-Hourly", engine="zarr")
#         imerge = imerge["GEOS-5_FWI"]
#         imerge.rio.write_crs("epsg:4326", inplace=True)
#         imerge.rio.set_spatial_dims(x_dim = "lon", y_dim = "lat", inplace = True)
        

    ## Get imerge -- Need to do on whole area? How manke more computationally efficient?
    final_perimeter = max(gdf[gdf.t == max(gdf.t)].geometry)
    #print(final_perimeter.envelope.exterior.coords.xy)
    lons = final_perimeter.envelope.exterior.coords.xy[0]
    lats = final_perimeter.envelope.exterior.coords.xy[1]
    img_clip = imerge.rio.clip_box(minx = min(lons), miny = min(lats), maxx=max(lons), maxy = max(lats), auto_expand= True)
    
    if(add_anomolies):
        img_clip = fv.imerge_climate(img_clip ,clim = ["rank", "anomolie","rank_anomolie"], var = ["FWI"])
    
    img_clip = img_clip.sel(time = slice(min(gdf.t), max(gdf.t))).mean(dim = ["lat", "lon"])
    img_clip = img_clip.to_dataframe()
    img_clip.drop(columns = "spatial_ref", inplace = True)
    img_clip.dropna(inplace = True)
    
    #gdf["fireID"] = id
    gdf = gdf.rename(columns = {"t" : "time"})
    gdf['time'] = gdf['time'].astype('datetime64[ns]')
    full = pd.merge(gdf,img_clip, on = "time", how = "outer")
    full = full.rename(columns = {"time": "t"}) ## Seems silly but I cant get the img_clip to convert it's name
    
    return(full)

In [ ]:
## Make into a function

def get_gpm_spread(full, pct_max_spread = 0.20):
    '''
    Finds "spread_days" and "non_spread_days" for fire. 
    
    INPUTS:
        
        full (GeoDataFrame): GDF with farea. Output from Imerge-merge
        pct_max_spread (float): Percentage of biggest increase in fire area that should count as a "spread day"
    
    '''
    
    cols = list(full.columns.values)
    
    full_d = full[full.t.dt.hour > 0] ## Daytime
    full_d = full_d.sort_values(by = ["t"]) #--- Need to sort by time, will otherwise calcualete diffs based on weird sorting
    full_n = full[full.t.dt.hour < 12] ## Nighttime
    full_n = full_n.sort_values(by = ["t"])

    fulls = [full_n, full_d]
    labels = ["Night", "Day"]

    for f,l in zip(fulls, labels):
    
        f["fline_diff"] = f.flinelen.diff()
        f["farea_diff"] = f.farea.diff()

        max_spread = max(f.fline_diff[f.fline_diff.notna()]) ## by calculating a different max for daytime vs nighttime, can get a step change in one that doesn't qualify, but would qaulify for the other. 
        
        f["spread_line"] = ((f["farea_diff"] > 0) & (f["farea_diff"] > (max_spread * pct_max_spread)))
        f.spread_line[f.t == min(f.t)] = 2 # First detection need special catagory. Maybe always counts as spread day, maybe "other"
        f["spread_line" + l] = f.spread_line
        #f.rename(columns = {"spread_line": "spread_line" + l})
        #plt.scatter(f.t, f.farea, c = f.spread_line)
    
    cols = cols + ["spread_line", "farea_diff", "fline_diff"]
    fire = pd.merge(full_n,full_d, on = cols, how = "outer")
    fire = fire.sort_values(by = ["t"])
    
    max_spread = max(fire.fline_diff[fire.fline_diff.notna()]) ## Recalulating max and spread days for overall maximum. 
    fire["spread_day"] = ((fire["farea_diff"] > 0) & (fire["farea_diff"] > (max_spread * pct_max_spread)))
    fire.spread_day[fire.t == min(fire.t)] = 2
    
    return(fire)

def get_spread_period(full_fr, time_of_interest = 12,  hours_before = 0, lags = [-2,-4,-12,-24,-48]):
    
    lables = ["Night", "Day"]
    for i in lables:

        times = full_fr["t"].loc[full_fr["spread_line" + i] == True] ## Is this acurate given the definitions above? 
        times_not = full_fr["t"].loc[full_fr["spread_line" + i] == False]
        #times_not = times_not[(times_not.dt.hour == 13) & (times_not.dt.minute == 30)]

        full_fr["spread_period"+ i] = np.nan

        for t in times:
            times_start = t - np.timedelta64((time_of_interest + hours_before), "h")
            times_end = t - np.timedelta64((hours_before), "h")
            full_fr["spread_period"+ i][(full_fr["t"] >= times_start) & (full_fr["t"] <= times_end)] = True


        for l in times_not:
            times_start = l - np.timedelta64((time_of_interest + hours_before), "h")
            times_end = l - np.timedelta64((hours_before), "h")
            full_fr["spread_period"+ i][(full_fr["t"] >= times_start) & (full_fr["t"] <= times_end)] = False

        for lag in lags:
            full_fr["spread_period"+ i +"_lag_" + str(lag)] = full_fr["spread_period"+ i].shift(lag)

    return(full_fr)

    

In [ ]:
#### Function in horrible for loop ####

def spread_process(fireID, year, path_region):
    i = fireID
    
    # fire cols 
    fire_cols = ['n_pixels', 
         'n_newpixels', 
         'farea', 
         'fperim', 
         'flinelen', 
         'duration',
         'pixden', 
         'meanFRP', 
         't', 
         'geometry', 
         'lon', 
         'lat', 
         'fireID',
         'fline_diff', 
         'farea_diff', 
         'spread_line', 
         'spread_lineNight',
         'spread_lineDay', 
         'spread_day', 
         'GEOS-5_FWI', 
         'spread_periodNight',
         'spread_periodNight_lag_-2', 
         'spread_periodNight_lag_-4',
         'spread_periodNight_lag_-12', 
         'spread_periodNight_lag_-24',
         'spread_periodNight_lag_-48', 
         'spread_periodDay',
         'spread_periodDay_lag_-2', 
         'spread_periodDay_lag_-4',
         'spread_periodDay_lag_-12', 
         'spread_periodDay_lag_-24',
         'spread_periodDay_lag_-48', "func_failed", "log"]
    
    try:
        raw_fire = load_large_fire(i, year = year, path_region= path_region)
        full = get_gpm_spread(raw_fire)
    except Exception as e:
        print("Error at load large fire/ get_gpm with Fire: ",i,)
        bad_log = traceback.format_exc(limit = 40) # Get log of error
        print(bad_log)
        
        fire = pd.DataFrame(np.array([[np.nan]*len(fire_cols)]), columns = fire_cols)
        fire["fireID"] = i
        fire["func_failed"] = "load_large_fire/get_gpm"
        fire["log"] = bad_log   
    else:
        try:
            uncorrected_fire = imerge_merge(full, imerge)
            fire = get_spread_period(uncorrected_fire)
        except Exception as e:
            print("Error in imerge merge or get_sprea_period for fire: ",i)
            bad_log = traceback.format_exc(limit = 40) # Get log of error
            print(bad_log)
            fire = pd.DataFrame(np.array([[np.nan]*len(fire_cols)]), columns = fire_cols)
            fire["fireID"] = i
            fire["func_failed"] = "load_large_fire/get_gpm"
            fire["log"] = bad_log

    return(fire)

In [30]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=1, n_workers=3)

#client.cluster.scale(3) 

client

/projects/myenvs/fireatlas_oct4/lib/python3.8/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33839 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:33839/status,
Dashboard: http://127.0.0.1:33839/status,Workers: 3
Total threads: 3,Total memory: 4.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33143,Workers: 3
Dashboard: http://127.0.0.1:33839/status,Total threads: 3
Started: Just now,Total memory: 4.00 GiB
Comm: tcp://127.0.0.1:41865,Total threads: 1
Dashboard: http://127.0.0.1:38531/status,Memory: 1.33 GiB
Nanny: tcp://127.0.0.1:33611,


In [31]:
### Put all together into horrible for-loop
#small_ids = ["7656", "8335"]
#small_ids = file_ids
#small_ids = ids.astype("str")
#small_ids = ids.astype("str")
#small_ids = ['13203', '12959', '13051', '10491', '12984']
#small_ids = ['13203', '12959', '13051', '10491', '12985']
# small_ids = ['13203', '12959', '13051', '10491', '12984', '12961', '12953', '10532', '12981', '10523', 
#              '13132', '13066', '13201', '13069', '12941', '12380', '12963', '13042',
#             '12985', '11440', '10261', '13525']
#small_ids = ["12985"]
small_ids = list(file_ids)

import time
from datetime import date

#import logging
#logging.basicConfig(level=logging.ERROR)
#logger = logging.getLogger(__name__)


# Grab Currrent Time Before Running the Code
start = time.time()

################# Horrible for loop  FUNCTION ##################
sub_region = "all"
region =  "WesternUS_REDO"
year = "2021"

run_name = str(date.today()) + "dask_test"
allfr = [] # list of fire timeseries
#bad_ids = [] # list of IDs that failed due to error



for n,i in enumerate(small_ids, start = 0):
    
    fire = client.submit(spread_process, i, year, region)
    #fire = spread_process(i, year, region)
    #fire = spread_process(i, year, region)
    allfr.append(fire)

/tmp/ipykernel_317/1849022387.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/tmp/ipykernel_317/1849022387.py:23: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/tmp/ipykernel_317/1014166134.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_317/1014166134.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_317/1

In [32]:
allfr

[<Future: finished, type: geopandas.geodataframe.GeoDataFrame, key: spread_process-ffe75ec7702e40993cef80306bce0721>,
 <Future: finished, type: geopandas.geodataframe.GeoDataFrame, key: spread_process-4a24723a7a51f0cc19b77cec34af3764>,
 <Future: finished, type: geopandas.geodataframe.GeoDataFrame, key: spread_process-e5ec0ce4429c219841923ae110330465>,
 <Future: finished, type: geopandas.geodataframe.GeoDataFrame, key: spread_process-c3550007380e693a409435660740b616>,
 <Future: finished, type: geopandas.geodataframe.GeoDataFrame, key: spread_process-e0c2874f05bbdf5fe0598069beae5680>,
 <Future: finished, type: geopandas.geodataframe.GeoDataFrame, key: spread_process-ebcde082c75d2b64d53235915f2b050d>,
 <Future: finished, type: geopandas.geodataframe.GeoDataFrame, key: spread_process-b9fde6bff89503899321c9c26cd45094>,
 <Future: finished, type: geopandas.geodataframe.GeoDataFrame, key: spread_process-811d013c8887e24361cfe10ddb4c74f9>,
 <Future: finished, type: pandas.core.frame.DataFrame, k

In [33]:
allfr_2 = client.gather(allfr)
#allfr = dask.compute(*allfr)

In [35]:
manyfr = gpd.GeoDataFrame(pd.concat(allfr_2, ignore_index=True)) # , crs = "EPSG:4326"
manyfr.to_csv("/projects/my-public-bucket/fire_spread/" + region + "_" + sub_region + "_" + year + "_" + run_name +".csv")

#debug = pd.DataFrame(bad_ids)
#debug.to_csv("/projects/my-public-bucket/fire_spread/DEBUG_" + region + "_" + sub_region + "_" + year + "_" + run_name + ".csv") ## Need to generate custom path

# Grab Currrent Time After Running the Code
end = time.time()

#Subtract Start Time from The End Time
total_time = end - start
print("\n"+ str(total_time))

In [36]:
manyfr

/projects/myenvs/fireatlas_oct4/lib/python3.8/site-packages/IPython/lib/pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
/projects/myenvs/fireatlas_oct4/lib/python3.8/site-packages/IPython/core/formatters.py:343: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,n_pixels,n_newpixels,farea,fperim,flinelen,duration,pixden,meanFRP,t,geometry,...,spread_periodNight_lag_-4,spread_periodNight_lag_-12,spread_periodNight_lag_-24,spread_periodNight_lag_-48,spread_periodDay,spread_periodDay_lag_-2,spread_periodDay_lag_-4,spread_periodDay_lag_-12,spread_periodDay_lag_-24,spread_periodDay_lag_-48
0,1.0,1.0,0.141,1.177624,0.0,0.0,7.092199,0.57,2021-07-06 00:00:00,"POLYGON ((-102.07779 31.79924, -102.07779 31.7...",...,False,False,False,False,NaN,False,False,True,False,False
1,1.0,0.0,0.141,1.177624,0.0,0.0,7.092199,0.00,2021-07-06 12:00:00,"POLYGON ((-102.07779 31.79924, -102.07779 31.7...",...,False,False,False,False,NaN,False,False,True,False,False
2,1.0,0.0,0.141,1.177624,0.0,0.0,7.092199,0.00,2021-07-07 00:00:00,"POLYGON ((-102.07779 31.79924, -102.07779 31.7...",...,False,False,False,False,False,False,False,False,False,False
3,1.0,0.0,0.141,1.177624,0.0,0.0,7.092199,0.00,2021-07-07 12:00:00,"POLYGON ((-102.07779 31.79924, -102.07779 31.7...",...,False,False,False,False,False,False,False,False,False,False
4,1.0,0.0,0.141,1.177624,0.0,0.0,7.092199,0.00,2021-07-08 00:00:00,"POLYGON ((-102.07779 31.79924, -102.07779 31.7...",...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# ### Test mega function without dask

# small_ids = ['13203', '12959', '13051', '10491', '12984']
# # small_ids = ['13203', '12959', '13051', '10491', '12984', 
# #              '12961', '12953', '10532', '12981', '10523',
# #             '13132', '13066', '13201', '13069', '12941', 
# #              '12380', '12963', '13042',
# #             '12985', '11440', '10261', '13525']

# #small_ids = ["12985"]



# import time
# from datetime import date


# # Grab Currrent Time Before Running the Code
# start = time.time()

# ################# Horrible for loop  FUNCTION ##################
# sub_region = "PNW"
# region =  "WesternUS_REDO"
# year = "2020"

# run_name = str(date.today()) + "dask_test"
# allfr = [] # list of fire timeseries
# #bad_ids = [] # list of IDs that failed due to error

# for n,i in enumerate(small_ids, start = 0):
    
#     fire = spread_process(i, year, region)
#     #fire = dask.delayed(spread_process)(i, year, region)
#     allfr.append(fire)


# #allfr = dask.compute(*allfr)
# manyfr = gpd.GeoDataFrame(pd.concat(allfr, ignore_index=True) ) #crs = "EPSG:4326"
# manyfr.to_csv("/projects/my-public-bucket/fire_spread/" + region + "_" + sub_region + "_" + year + "_" + run_name +".csv")

# #debug = pd.DataFrame(bad_ids)
# #debug.to_csv("/projects/my-public-bucket/fire_spread/DEBUG_" + region + "_" + sub_region + "_" + year + "_" + run_name + ".csv") ## Need to generate custom path

# # Grab Currrent Time After Running the Code
# end = time.time()

# #Subtract Start Time from The End Time
# total_time = end - start
# print("\n"+ str(total_time))

/tmp/ipykernel_5621/1849022387.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lon'] = gdf.centroid.x
/tmp/ipykernel_5621/1849022387.py:23: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y
/tmp/ipykernel_5621/1849022387.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lon'] = gdf.centroid.x
/tmp/ipykernel_5621/1849022387.py:23: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf


100.45326447486877


In [ ]:
# ## Dask delayed time
# import dask

# #small_ids = ids.astype("str")
# small_ids = ['13203', '12959', '13051', '10491', '12984']

# import time
# from datetime import date

# #import logging
# #logging.basicConfig(level=logging.ERROR)
# #logger = logging.getLogger(__name__)


# # Grab Currrent Time Before Running the Code
# start = time.time()

# ################# DASK Horrible for loop ##################
# sub_region = "PNW"
# region =  "WesternUS_REDO"
# year = "2020"

# run_name = str(date.today()) + "dask_delayed_regional"
# allfr = [] # list of fire timeseries
# bad_ids = [] # list of IDs that failed due to error

# for n,i in enumerate(small_ids, start = 0):
#     #print(n)
#     #print(i)

# #     full = imerge_merge(i, "2020", "WesternUS")
# #     fire = get_gpm_spread(full)

# #     #fire = pd.Data
# #     allfr.append(fire)
        
#     try:
#         #params = {'fireID':[i], 'year':[year], 'path_region':[region]}
#         #params = pd.DataFrame.from_dict(params, orient='columns')
        
#         raw_fire = dask.delayed(load_large_fire)(i, year, region)
#         full = dask.delayed(get_gpm_spread)(raw_fire)
#     except Exception as e:
#         print("Error at load_large_fire with Fire: ",i,)
#         bad_log = traceback.format_exc(limit = 40) # Get log of error
#         print(bad_log)
#         bad_ids.append(
#         {
#             "fireID": i, 
#             "func_failed": "imerge_merge",
#             "log": bad_log
#         }
#         )
#         #logger.debug(traceback.format_exc())
#         next
#     else:
#         try:
#             uncorrected_fire = dask.delayed(imerge_merge)(full)
#             #print(uncorrected_fire.fireID)
#             fire = dask.delayed(get_spread_period)(uncorrected_fire)
#             #print(fire.fireID)
#         except Exception as e:
#             print("Error in imerge_merge or calculating spread days in Fire: ",i)
#             bad_log = traceback.format_exc(limit = 40) # Get log of error
#             print(bad_log)
#             bad_ids.append(
#             {
#                 "fireID": i, 
#                 "func_failed": "get_gpm_spread",
#                 "log": bad_log
#             }
#             )
#             #logger.debug(traceback.format_exc())
#             next
#         #fire = pd.Data
#         allfr.append(fire)
# allfr = dask.compute(*allfr)
# manyfr = gpd.GeoDataFrame(pd.concat(allfr, ignore_index=True), crs= fire.crs)
# manyfr.to_csv("/projects/my-public-bucket/fire_spread/" + region + "_" + sub_region + "_" + year + "_" + run_name +".csv")

# debug = pd.DataFrame(bad_ids)
# debug.to_csv("/projects/my-public-bucket/fire_spread/DEBUG_" + region + "_" + sub_region + "_" + year + "_" + run_name + ".csv") ## Need to generate custom path

# # Grab Currrent Time After Running the Code
# end = time.time()

# #Subtract Start Time from The End Time
# total_time = end - start
# print("\n"+ str(total_time))

In [184]:
# ### Put all together into horrible for-loop
# #small_ids = ["7656", "8335"]
# #small_ids = file_ids
# #small_ids = ids.astype("str")
# #small_ids = ids.astype("str")
# #small_ids = ['13203', '12959', '13051', '10491', '12984']
# small_ids = ['13203']#, '12959', '13051', '10491', '12984']

# import time
# from datetime import date

# #import logging
# #logging.basicConfig(level=logging.ERROR)
# #logger = logging.getLogger(__name__)


# # Grab Currrent Time Before Running the Code
# start = time.time()

# ################# Horrible for loop ##################
# sub_region = "PNW"
# region =  "WesternUS_REDO"
# year = "2020"

# run_name = str(date.today()) + "region_workflow_hourly"
# allfr = [] # list of fire timeseries
# bad_ids = [] # list of IDs that failed due to error

# for n,i in enumerate(small_ids, start = 0):
#     #print(n)
#     #print(i)

# #     full = imerge_merge(i, "2020", "WesternUS")
# #     fire = get_gpm_spread(full)

# #     #fire = pd.Data
# #     allfr.append(fire)
        
#     try:
#         raw_fire = load_large_fire(i, year = year, path_region= region)
#         #raw_fire["fireID"] = str(i)
#         full = get_gpm_spread(raw_fire)
#     except Exception as e:
#         print("Error at imerge_merge with Fire: ",i,)
#         bad_log = traceback.format_exc(limit = 40) # Get log of error
#         print(bad_log)
#         bad_ids.append(
#         {
#             "fireID": i, 
#             "func_failed": "imerge_merge",
#             "log": bad_log
#         }
#         )
#         #logger.debug(traceback.format_exc())
#         next
#     else:
#         try:
#             uncorrected_fire = imerge_merge(full)
#             #print(uncorrected_fire.fireID)
#             fire = get_spread_period(uncorrected_fire)
#             #print(fire.fireID)
#         except Exception as e:
#             print("Error in calculating spread days in Fire: ",i)
#             bad_log = traceback.format_exc(limit = 40) # Get log of error
#             print(bad_log)
#             bad_ids.append(
#             {
#                 "fireID": i, 
#                 "func_failed": "get_gpm_spread",
#                 "log": bad_log
#             }
#             )
#             #logger.debug(traceback.format_exc())
#             next
#         #fire = pd.Data
#         allfr.append(fire)
#     if((n%5 == 0) | (n == (len(small_ids) -1))):
#         print("")
#         manyfr = gpd.GeoDataFrame(pd.concat(allfr, ignore_index=True), crs= fire.crs)
#         manyfr.to_csv("/projects/my-public-bucket/fire_spread/" + region + "_" + sub_region + "_" + year + "_" + run_name +".csv")
        
#         debug = pd.DataFrame(bad_ids)
#         debug.to_csv("/projects/my-public-bucket/fire_spread/DEBUG_" + region + "_" + sub_region + "_" + year + "_" + run_name + ".csv") ## Need to generate custom path

# # Grab Currrent Time After Running the Code
# end = time.time()

# #Subtract Start Time from The End Time
# total_time = end - start
# print("\n"+ str(total_time))